<a href="https://colab.research.google.com/github/claredavies/InterviewTaskDataCleaning/blob/main/InterviewDataPrepTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
from google.colab import files
import pandas as pd
from matplotlib import pyplot as plt
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
import re
import socket
import datetime
from datetime import datetime

In [2]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows', None)

# Read in files

In [3]:
!git clone https://github.com/claredavies/InterviewTaskDataCleaning.git

Cloning into 'InterviewTaskDataCleaning'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 49 (delta 26), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [4]:
%cd InterviewTaskDataCleaning

/content/InterviewTaskDataCleaning


In [5]:
df = pd.read_csv('InputData.csv')
df.head(3)

,Ref,First_Name,Last_Name,Email,Gender,IP_Address,Date_of_Birth
0,1,Germayne,Simmons,gsimmons0@cdbaby.com,Male,24.48.223.251,18/11/1959
1,2,Cosmo,Pleass,cpleass1@netscape.com,Male,108.87.136.28,08/11/1935
2,3,Henrik,Boag,hboag2@bizjournals.com,Male,221.105.60.101,07/10/1989


# Validation Functions

In [6]:
def boolCheckLengthOverLimitString(value, limit):
  lengthFound = len(value)
  if(lengthFound > limit):
    return True
  else:
    return False

In [7]:
def boolCheckLengthOverLimitInt(value, limit):
  stringValue = str(value)
  return boolCheckLengthOverLimitString(stringValue, limit)

In [8]:
def checkFirstLetterCapitalised(string):
  lengthWord = len(string)
  return string[0].isupper()

In [9]:
def containsNumber(string):
    return any(char.isdigit() for char in string)

In [10]:
def checkIfSpacesBetweenWords(string):
  res = bool(re.search(r"\s", string))
  return res

In [11]:
def checkMaleOrFemale(string):
  lowercaseString = string.lower()
  if lowercaseString == 'female':
    return True
  elif lowercaseString == 'male':
    return True
  else:
    return False

In [12]:
def checkIfIPV4Valid(string):
  try:
      socket.inet_aton(string)
      return True
      # legal
  except socket.error:
    return False

In [13]:
def checkFormatDate(string):
  format = '%d/%m/%Y'
  try:
    datetime.strptime(string, format)
    return True
  except ValueError:
    return False 

In [14]:
def check18OrOverBy1stDec2020(startDate):
  endDate = '01/12/2020'
  endDate = datetime.strptime(endDate, '%d/%m/%Y')
  startDate = datetime.strptime(startDate, '%d/%m/%Y')
  yearsOld = endDate.year - startDate.year - ((endDate.month, endDate.day) < (startDate.month, startDate.day))
  if yearsOld >= 18:
    return True
  else:
    return False

In [15]:
def createHTMLOfDf(df, name):
  html = df.to_html()
  
  # write html to file
  text_file = open(name, "w")
  text_file.write(html)
  text_file.close()

In [16]:
messageErrorNA = "Value is NaN"
messageErrorOverLength = "Length is over the limit"
messageErrorCaseIncorrect = "Case is incorrect"
messageErrorContainsUnwantedNumbers = "Unwanted numbers contained"
messageErrorUnwantedSpace = "Unwanted space"
messageErrorNotMaleOrFemale = "Not Male or Female"
messageErrorNotString = "Not a string"
messageErrorNotValidIPAddress = "Not a valid IPv4"
messageErrorNot18OrOverBy1stDec2020 = "Not 18 or over by 1st Dec 2020"
messageDateOfBirthIncorrectFormat = "Date of birth not in format dd/mm/yyyy"

# Check Duplicates and Size

In [17]:
print(df.shape)

(1000, 7)


In [18]:
dfWithoutRef = df.drop('Ref', axis=1)
dfWithoutRef.head(3)

,First_Name,Last_Name,Email,Gender,IP_Address,Date_of_Birth
0,Germayne,Simmons,gsimmons0@cdbaby.com,Male,24.48.223.251,18/11/1959
1,Cosmo,Pleass,cpleass1@netscape.com,Male,108.87.136.28,08/11/1935
2,Henrik,Boag,hboag2@bizjournals.com,Male,221.105.60.101,07/10/1989


In [19]:
duplicate = df[dfWithoutRef.duplicated()]
print("Duplicate Rows :")
print(duplicate)

Duplicate Rows :
Empty DataFrame
Columns: [Ref, First_Name, Last_Name, Email, Gender, IP_Address, Date_of_Birth]
Index: []


# Check Ref Column

In [20]:
# want to add in check for unique
def checkRefColumn(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if boolCheckLengthOverLimitInt(input, 8) == True:
    issueMessage = messageErrorOverLength
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  elif pd.isna(input):
    issueMessage = messageErrorNA
    dfFlaws = dfFlaws.append({'Ref': ref, 'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check First Name

In [21]:
def checkColumnFirstName(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.isna(input):
    issueMessage = messageErrorNA
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    if boolCheckLengthOverLimitInt(input, 20) == True:
      issueMessage = messageErrorOverLength
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)


    if(checkFirstLetterCapitalised(input) == False):
      issueMessage = messageErrorCaseIncorrect
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    if checkIfSpacesBetweenWords(input) == True:
      issueMessage = messageErrorUnwantedSpace
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    # need to search for numbers within
    if containsNumber(input) == True:
      issueMessage = messageErrorContainsUnwantedNumbers
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check Second Name

In [22]:
def checkColumnSecondName(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.isna(input):
    issueMessage = messageErrorNA
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    if boolCheckLengthOverLimitInt(input, 40) == True:
      issueMessage = messageErrorOverLength
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    if(checkFirstLetterCapitalised(input) == False):
      issueMessage = messageErrorCaseIncorrect
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)


    # need to search for numbers within
    if containsNumber(input) == True:
      issueMessage = messageErrorContainsUnwantedNumbers
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    if checkIfSpacesBetweenWords(input) == True:
      issueMessage = messageErrorUnwantedSpace
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check Email 

In [23]:
def checkColumnEmail(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if boolCheckLengthOverLimitInt(input, 254) == True:
    issueMessage = messageErrorOverLength
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  return dfFlaws

# Check Gender

In [24]:
def checkColumnGender(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(input):
    if type(input) == str:
      if boolCheckLengthOverLimitInt(input, 6) == True:
        issueMessage = messageErrorOverLength
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

      if checkMaleOrFemale(input) == False:
        issueMessage = messageErrorNotMaleOrFemale
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
    
      if checkIfSpacesBetweenWords(input) == True:
        issueMessage = messageErrorUnwantedSpace
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
    else:
      issueMessage = messageErrorNotString
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  else:
      issueMessage = messageErrorNA
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check IP Address

In [25]:
def checkColumnIPAddress(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(input):
    if type(input) == str:

      if boolCheckLengthOverLimitInt(input, 15) == True:
        issueMessage = messageErrorOverLength
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      if checkIfIPV4Valid(input) == False:
        issueMessage = messageErrorNotValidIPAddress 
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

    else:
      issueMessage = messageErrorNotString
      dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    issueMessage = messageErrorNA
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Check Date of Birth

In [26]:
def checkColumnDateOfBirth(input, columnName, ref):
  dfFlaws = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  if pd.notna(input):
    if type(input) == str:

      if boolCheckLengthOverLimitInt(input, 10) == True:
        issueMessage = messageErrorOverLength
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      if checkFormatDate(input) == False:
        issueMessage = messageDateOfBirthIncorrectFormat
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
      
      else:
        if check18OrOverBy1stDec2020(input) == False:
           issueMessage = messageErrorNot18OrOverBy1stDec2020
           dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
    else:
        issueMessage = messageErrorNotString
        dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)

  else:
    issueMessage = messageErrorNA
    dfFlaws = dfFlaws.append({'Ref': ref,'ColumnName': columnName, 'ColumnValue': input, 'ColumnIssue': issueMessage, 'Fixed': False, 'FixedResult': 'N/A'}, ignore_index=True)
  
  return dfFlaws

# Find List of Errors

In [167]:
def findErrorsData(dataframe):

  # check all values unique
  print("Are ref unique: " + str(dataframe['Ref'].is_unique))

  # check each row
  dfErrorsFound = pd.DataFrame(columns=['Ref', 'ColumnName', 'ColumnValue', 'ColumnIssue','Fixed',"FixedResult"])

  for index, row in dataframe.iterrows():
    dfErrorsFoundRefColumn = checkRefColumn(row["Ref"], "Ref", row["Ref"])
    if dfErrorsFoundRefColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundRefColumn, ignore_index=True)

    dfErrorsFoundFirstNameColumn = checkColumnFirstName(row["First_Name"], "First_Name", row["Ref"])
    if dfErrorsFoundFirstNameColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundFirstNameColumn, ignore_index=True)

    dfErrorsFoundLastNameColumn = checkColumnSecondName(row["Last_Name"], "Last_Name", row["Ref"])
    if dfErrorsFoundLastNameColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundLastNameColumn, ignore_index=True)

    dfErrorsFoundEmailColumn = checkColumnEmail(row["Email"], "Email", row["Ref"])
    if dfErrorsFoundEmailColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundEmailColumn, ignore_index=True)

    dfErrorsFoundGenderColumn = checkColumnGender(row["Gender"], "Gender", row["Ref"])
    if dfErrorsFoundGenderColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundGenderColumn, ignore_index=True)

    dfErrorsFoundIPAddressColumn = checkColumnIPAddress(row["IP_Address"], "IP_Address", row["Ref"])
    if dfErrorsFoundIPAddressColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundIPAddressColumn, ignore_index=True)

    dfErrorsFoundDateOfBirthColumn = checkColumnDateOfBirth(row["Date_of_Birth"], "Date_of_Birth", row["Ref"])
    if dfErrorsFoundDateOfBirthColumn.empty == False:
      dfErrorsFound = dfErrorsFound.append(dfErrorsFoundDateOfBirthColumn, ignore_index=True)
  
  return dfErrorsFound

In [168]:
dfErrors = findErrorsData(df)

Are ref unique: True


In [169]:
createHTMLOfDf(dfErrors, "DfErrors.html")

In [170]:
print(dfErrors.head())

  Ref  ColumnName                              ColumnValue  \
0   6  First_Name                               christabel   
1   6   Last_Name                                    start   
2   9  IP_Address  1df6:b3b9:cdb7:d246:3fe9:7288:46ee:e528   
3   9  IP_Address  1df6:b3b9:cdb7:d246:3fe9:7288:46ee:e528   
4  27   Last_Name                                      NaN   

                ColumnIssue  Fixed FixedResult  
0         Case is incorrect  False         N/A  
1         Case is incorrect  False         N/A  
2  Length is over the limit  False         N/A  
3          Not a valid IPv4  False         N/A  
4              Value is NaN  False         N/A  


# Data Error Exploration

In [171]:
def inspectRowsEffectedByError(dfOriginal, dfErrors, errorMessage):
  noRowsOriginalDf = len(dfOriginal.index)
  errorDfSpecificErrorMsg = dfErrors[dfErrors['ColumnIssue'] == errorMessage]
  print("where: " + errorMessage)
  noRowsErrorDfSpecificErrorMsg = len(errorDfSpecificErrorMsg.index)
  percentage = (noRowsErrorDfSpecificErrorMsg/noRowsOriginalDf)*100
  percentage = str(round(percentage, 2))
  print("% where : " + errorMessage + "  " + percentage + "%") 
  print()
  return errorDfSpecificErrorMsg

Gender

In [172]:
errorDfSpecificErrorMsg = dfErrors[dfErrors['ColumnIssue'] == messageErrorNotMaleOrFemale]
print(errorDfSpecificErrorMsg)

    Ref ColumnName ColumnValue         ColumnIssue  Fixed FixedResult
14  124     Gender           1  Not Male or Female  False         N/A
18  158     Gender          m   Not Male or Female  False         N/A
35  347     Gender           1  Not Male or Female  False         N/A
37  354     Gender           1  Not Male or Female  False         N/A
39  402     Gender        fem   Not Male or Female  False         N/A
52  521     Gender           2  Not Male or Female  False         N/A
58  586     Gender          f   Not Male or Female  False         N/A


Containing Numbers

In [173]:
errorDfSpecificErrorMsg = dfErrors[dfErrors['ColumnIssue'] == messageErrorContainsUnwantedNumbers]
print(errorDfSpecificErrorMsg)

genderIs1ain = df[df['First_Name'] == '1ain']
print(genderIs1ain)

    Ref  ColumnName ColumnValue                 ColumnIssue  Fixed FixedResult
28  298   Last_Name     Gambe11  Unwanted numbers contained  False         N/A
47  482  First_Name        1ain  Unwanted numbers contained  False         N/A
55  554  First_Name        0wen  Unwanted numbers contained  False         N/A
83  815   Last_Name  0'Dowgaine  Unwanted numbers contained  False         N/A
     Ref First_Name Last_Name                    Email Gender     IP_Address  \
481  482       1ain    Freeth  ifreethdd@artisteer.com   Male  162.225.238.5   

    Date_of_Birth  
481    03/10/1935  


Date

In [174]:
errorDfSpecificErrorMsg = dfErrors[dfErrors['ColumnIssue'] == messageDateOfBirthIncorrectFormat]
print(errorDfSpecificErrorMsg)

     Ref     ColumnName  ColumnValue                             ColumnIssue  \
5     36  Date_of_Birth   01/31/1995  Date of birth not in format dd/mm/yyyy   
10   102  Date_of_Birth   27/27/1991  Date of birth not in format dd/mm/yyyy   
90   878  Date_of_Birth  09-MAY-1956  Date of birth not in format dd/mm/yyyy   
101  965  Date_of_Birth  06/05/1943*  Date of birth not in format dd/mm/yyyy   

     Fixed FixedResult  
5    False         N/A  
10   False         N/A  
90   False         N/A  
101  False         N/A  


In [175]:
inspectRowsEffectedByError(df, dfErrors, messageErrorNA)

inspectRowsEffectedByError(df, dfErrors, messageErrorOverLength)

inspectRowsEffectedByError(df, dfErrors, messageErrorNotValidIPAddress)

inspectRowsEffectedByError(df, dfErrors, messageErrorNot18OrOverBy1stDec2020)

inspectRowsEffectedByError(df, dfErrors, messageErrorNotString)

where: Value is NaN
% where : Value is NaN  0.7%

where: Length is over the limit
% where : Length is over the limit  0.6%

where: Not a valid IPv4
% where : Not a valid IPv4  0.6%

where: Not 18 or over by 1st Dec 2020
% where : Not 18 or over by 1st Dec 2020  4.5%

where: Not a string
% where : Not a string  0.0%



,Ref,ColumnName,ColumnValue,ColumnIssue,Fixed,FixedResult


# Clean Functions

In [176]:
def removeSpace(value):
  valueWithoutSpace = value.replace(" ", "")
  return valueWithoutSpace

In [177]:
def fixSpace(df, ref, columnName):
  row = df[df['Ref'] == ref]
  rowValueColumnName = row[columnName].iat[0]
  rowValueColumnNameWithoutSpace = removeSpace(rowValueColumnName)
  df.loc[df['Ref'] == ref,[columnName]] = rowValueColumnNameWithoutSpace
  return df

In [178]:
def fixNumbersToLetters(df, ref, columnName):
  row = df[df['Ref'] == ref]
  rowValueColumnName = row[columnName].iat[0]
  rowValueColumnNameWithoutNumbers = convertNumbersToLetters(rowValueColumnName)
  df.loc[df['Ref'] == ref,[columnName]] = rowValueColumnNameWithoutNumbers
  return df

In [179]:
def convertNumbersToLetters(value):
  stringWithout0 = value.replace("0", "o" )
  stringWithout0And1 = stringWithout0.replace("1", "l" )
  return stringWithout0And1

In [180]:
def capitalismFirstLetter(df, ref, columnName):
  row = df[df['Ref'] == ref]
  rowValueColumnName = row[columnName].iat[0]
  rowValueColumnNameCapitalised = convertNumbersToLetters(rowValueColumnName)
  df.loc[df['Ref'] == ref,[columnName]] = rowValueColumnNameCapitalised.capitalize()
  return df

In [181]:
def replaceGenderTerms(value):
  value = str(value)
  if value == 'l':
    value = 'Female'
  elif value == '1':
    value = 'Female'
  elif value == '2':
    value = 'Male'
  elif value == 'm':
    value = 'Male'
  elif value == 'f':
    value = 'Female'
  elif value == 'fem':
    value = 'Female'
  return value

In [182]:
def fixGenderTerms(df, ref, columnName):
  row = df[df['Ref'] == ref]
  rowValueColumnName = row[columnName].iat[0]
  df.loc[df['Ref'] == ref,[columnName]] = replaceGenderTerms(rowValueColumnName)
  return df

In [183]:
def dateInFormMMDDYYYY(input):
  format = '%m/%d/%Y'
  try:
    datetime.strptime(input, format)
    return True
  except ValueError:
    return False 

In [184]:
def dateContainMonthName(input):
  format = '%d-%B-%Y'
  try:
    datetime.strptime(input, format)
    return True
  except ValueError:
    return False 

In [185]:
def convertDateMonthName(input):
  input = datetime.strptime(input, "%d-%B-%Y").strftime("%d/%m/%Y")
  return input

In [186]:
def dateConvertMMDDYYYYToDDMMYYYY(input):
  input = datetime.strptime(input, "%m/%d/%Y").strftime("%d/%m/%Y")
  return input

In [187]:
def checkIfContainsSpecialCharactersOtherThanSlash(input):
  special_characters = ['@','#','%', '*']
  return any(c in special_characters for c in input)

In [188]:
def removeSpecialCharactersOtherThanSlash(input):
  special_characters = ['@','#','%', '*']
  out_list = []
  for x in input:
    for y in special_characters:
        if y in x:
            input = input.replace(x,'')
  return input

In [189]:
def fixDate(df, ref, columnName):
  row = df[df['Ref'] == ref]
  rowValueColumnName = row[columnName].iat[0]
  if dateInFormMMDDYYYY(rowValueColumnName):
    df.loc[df['Ref'] == ref,[columnName]] = dateConvertMMDDYYYYToDDMMYYYY(rowValueColumnName)
  elif dateContainMonthName(rowValueColumnName):
    df.loc[df['Ref'] == ref,[columnName]] = convertDateMonthName(rowValueColumnName)
  elif checkIfContainsSpecialCharactersOtherThanSlash(rowValueColumnName):
    df.loc[df['Ref'] == ref,[columnName]] = removeSpecialCharactersOtherThanSlash(rowValueColumnName)
  return df

# Rows to Fix

In [190]:
def fixValues(df, dfErrors):
  dfClean = df.copy()

  # remove unwanted spaces
  errorDfSpaces = dfErrors[dfErrors['ColumnIssue'] == messageErrorUnwantedSpace]
  for index, row in errorDfSpaces.iterrows():
    dfClean = fixSpace(dfClean, row["Ref"], row["ColumnName"])

  # remove unwanted numbers
  errorDfUnwantedNumbers = dfErrors[dfErrors['ColumnIssue'] == messageErrorContainsUnwantedNumbers]
  for index, row in errorDfUnwantedNumbers.iterrows():
    dfClean = fixNumbersToLetters(dfClean, row["Ref"], row["ColumnName"])
  
  # fix case (first capital)
  errorDfCaseIncorrect = dfErrors[dfErrors['ColumnIssue'] == messageErrorCaseIncorrect]
  for index, row in errorDfCaseIncorrect.iterrows():
    dfClean = capitalismFirstLetter(dfClean, row["Ref"], row["ColumnName"])
  
  # fix gender
  errorDfUnwantedGenders = dfErrors[dfErrors['ColumnIssue'] == messageErrorNotMaleOrFemale]
  for index, row in errorDfUnwantedGenders.iterrows():
    dfClean = fixGenderTerms(dfClean, row["Ref"], row["ColumnName"])
  
  # fix date of birth format
  errorDfDateOfBirthFormat = dfErrors[dfErrors['ColumnIssue'] == messageDateOfBirthIncorrectFormat]
  for index, row in errorDfDateOfBirthFormat.iterrows():
    dfClean = fixDate(dfClean, row["Ref"], row["ColumnName"])
  
  return dfClean

In [191]:
cleanDf = fixValues(df, dfErrors)

In [192]:
print(cleanDf.head())

   Ref First_Name Last_Name                      Email  Gender  \
0    1   Germayne   Simmons       gsimmons0@cdbaby.com    Male   
1    2      Cosmo    Pleass      cpleass1@netscape.com    Male   
2    3     Henrik      Boag     hboag2@bizjournals.com    Male   
3    4    Kinsley   Millard         kmillard3@furl.net    Male   
4    5       Peri   Lippatt  plippatt4@liveinternet.ru  Female   

       IP_Address Date_of_Birth  
0   24.48.223.251    18/11/1959  
1   108.87.136.28    08/11/1935  
2  221.105.60.101    07/10/1989  
3   67.153.106.99    03/02/1954  
4   179.46.90.224    21/12/1988  


In [193]:
dfErrorsCleaned = findErrorsData(cleanDf)
createHTMLOfDf(dfErrorsCleaned, "DfErrorsFixErrorsCleaned.html")

Are ref unique: True


# Rows to Delete

In [194]:
def checkRowStillExists(df, ref):
  row = df[df['Ref'] == ref]
  if(len(row.index) > 0):
    return True
  else:
    return False

In [195]:
def dropRowBasedOnErrorMsg(df, dfErrors, errorMsg):
  errorDf = dfErrors[dfErrors['ColumnIssue'] ==  errorMsg]
  for index, row in errorDf.iterrows():
    if(checkRowStillExists(df, row["Ref"])):
      rowInDataframeToDelete = df[df["Ref"] == row["Ref"]]
      rowInDataframeToDeleteIndex = rowInDataframeToDelete.index
      df = df.drop(rowInDataframeToDeleteIndex)
  return df

In [196]:
def deleteUnwantedValuesDf(df, dfErrors):
  dfDeletedValues = df.copy()

  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageErrorNA)
  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageErrorOverLength)
  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageErrorNotValidIPAddress)
  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageErrorNot18OrOverBy1stDec2020)
  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageErrorNotString)
  dfDeletedValues = dropRowBasedOnErrorMsg(dfDeletedValues, dfErrors, messageDateOfBirthIncorrectFormat)

  return dfDeletedValues

In [197]:
cleanDeletedDf = deleteUnwantedValuesDf(cleanDf, dfErrorsCleaned)

In [198]:
dfErrorsCleanedDeleted = findErrorsData(cleanDeletedDf)
createHTMLOfDf(dfErrorsCleanedDeleted, "DfErrorsFixErrorsCleanedDeleted.html")

Are ref unique: True


In [199]:
createHTMLOfDf(cleanDeletedDf, "DfCleaned.html")

In [200]:
print("size after cleaning and deleting: " + str(len(cleanDeletedDf.index)))

size after cleaning and deleting: 942
